# Controlable Light Enhencement for Underater Image Enhancement with difusion model

### Download dataset

In [1]:
import os
import gdown

# Primeiro, instalar o gdown se ainda não estiver instalado
# !pip install gdown

# URL do Google Drive
url = "https://drive.google.com/file/d/1SCwOosZam8bzoZdVSwW60l-bD7c65pv0/view"

# Criar diretório para o dataset se não existir
os.makedirs("dataset", exist_ok=True)

# Baixar o arquivo
output = "dataset/dataset.zip"
gdown.download(url=url, output=output, fuzzy=True)

# Descompactar o arquivo
import zipfile
with zipfile.ZipFile(output, 'r') as zip_ref:
    zip_ref.extractall("dataset")

print("Dataset baixado e descompactado com sucesso!")

Downloading...
From (original): https://drive.google.com/uc?id=1SCwOosZam8bzoZdVSwW60l-bD7c65pv0
From (redirected): https://drive.google.com/uc?id=1SCwOosZam8bzoZdVSwW60l-bD7c65pv0&confirm=t&uuid=7a1705fb-c15d-4b96-a058-af475d87f4db
To: /home/pdi/Documents/UDBE/dataset/dataset.zip
100%|██████████| 914M/914M [00:20<00:00, 44.0MB/s] 


Dataset baixado e descompactado com sucesso!


### Download ckpt

In [9]:
# Import necessary libraries
import os
import requests
import zipfile

# Function to download and unzip the file
def download_and_unzip(url, extract_to):
    # Define the local zip file path
    zip_path = os.path.join(extract_to, 'output.zip')
    
    # Download the file
    print(f"Downloading from {url}...")
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(zip_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    file.write(chunk)
        print("Download complete.")
    else:
        print(f"Failed to download. HTTP Status Code: {response.status_code}")
        return
    
    # Unzip the file
    print("Extracting files...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print("Extraction complete.")
    
    # Remove the zip file
    os.remove(zip_path)
    print("Temporary zip file removed.")

# URL of the weights file
weights_url = "https://huggingface.co/Gusanagy/UDBE-Unsupervised-Diffusion-based-Brightness-Enhancement-in-Underwater-Images/resolve/main/UDBE.zip"

# Directory to save and extract the weights
weights_dir = 'output'
os.makedirs(weights_dir, exist_ok=True)

# Download and extract the weights
download_and_unzip(weights_url, weights_dir)

print("Weights downloaded and ready.")


Download complete.
Extracting files...
Extraction complete.
Temporary zip file removed.
Weights downloaded and ready.


### Inferência

### Teste dataset RUIE

In [ ]:
!python main.py --dataset "RUIE" --state "test" --pretrained_path "output/ckpt/RUIE/ckpt_1000_.pt" --wandb False 


### Teste dataset UIEB

In [1]:
!python main.py --dataset "UIEB" --state "val" --pretrained_path "output/ckpt/UIEB/ckpt_1000_.pt" --wandb False 

Namespace(dataset='UIEB', model='standart', dataset_path='./dataset/UDWdata/', state='val', pretrained_path='output/ckpt/UIEB/ckpt_1000_.pt', inference_image='data/UDWdata/UIEB/val/205_img_.png', output_path='./output/', wandb=True, wandb_name='GLDiffusion', epoch=1000, DDP=False, batch_size=16, T=1000, channel=128, channel_mult=[1, 2, 3, 4], attn=[2], num_res_blocks=2, dropout=0.15, lr=5e-05, multiplier=2.0, beta_1=0.0001, beta_T=0.02, img_size=32, grad_clip=1.0, device='cuda', device_list=[0, 1], ddim=True, unconditional_guidance_scale=1, ddim_step=100)
1000
Processando imagens: 100%|████████████████| 89/89 [00:00<00:00, 2785769.07img/s]


### Teste dataset SUIM

In [1]:
!python main.py --dataset "SUIM" --state "val" --pretrained_path "output/ckpt/SUIM/ckpt_1000_.pt" --wandb False 

Namespace(dataset='SUIM', model='standart', dataset_path='./dataset/UDWdata/', state='val', pretrained_path='output/ckpt/SUIM/ckpt_1000_.pt', inference_image='data/UDWdata/UIEB/val/205_img_.png', output_path='./output/', wandb=True, wandb_name='GLDiffusion', epoch=1000, DDP=False, batch_size=16, T=1000, channel=128, channel_mult=[1, 2, 3, 4], attn=[2], num_res_blocks=2, dropout=0.15, lr=5e-05, multiplier=2.0, beta_1=0.0001, beta_T=0.02, img_size=32, grad_clip=1.0, device='cuda', device_list=[0, 1], ddim=True, unconditional_guidance_scale=1, ddim_step=100)
1000
INICIANDO TEST
Tamanho do dataset : 110
Total test-training examples: 110
model load weight done.
savedir: ./output/result/SUIM/epoch/1000/, ckpt_path: output/ckpt/SUIM/ckpt_1000_.pt
Testing model: 100%|██████████████████████████| 110/110 [11:43<00:00,  6.40s/it]
Calculating advanced metrics...
Processing metrics: 100%|█████████████████████| 110/110 [00:04<00:00, 22.14it/s]
Métricas finais:
  PSNR: 24.0489
  SSIM: 0.8507
  UIQM: 